##TEAM MEMBERS

Aidan Stautland, 502841; Christian Torhaug, 564355; Jens Skaug, STUDID

In [1]:
import pandas as pd
from datetime import datetime

ais_sample_submission.csv

In [4]:
sampledf = pd.read_csv("ais_sample_submission.csv")

sampledf["latitude_predicted"] = sampledf["latitude_predicted"].apply(lambda x: float(x))
sampledf["longitude_predicted"] = sampledf["longitude_predicted"].apply(lambda x: float(x))

ais_test.csv

In [5]:
testdf = pd.read_csv("ais_test.csv", nrows=10) #nrows to speed up action

testdf["time"] = testdf["time"].apply(lambda x: datetime.strptime(x, "%Y-%m-%d %H:%M:%S"))
testdf["scaling_factor"] = testdf["scaling_factor"].apply(lambda x: float(x))

ais_train.csv

In [11]:
traindf = pd.read_csv("~/Ais_Data/ais_train.csv", sep="|", nrows=50000) #nrows to speed up action

def parseEta(row):
    try:
        return datetime.strptime(row['etaRaw'], "%m-%d %H:%M")
    except ValueError:
        return pd.NaT

#traindf["time"] = traindf["time"].apply(lambda x: datetime.strptime(x, "%Y-%m-%d %H:%M:%S"))
traindf["time"] = pd.to_datetime(traindf["time"]) 
traindf["cog"] = traindf["cog"].apply(lambda x: float(x) if float(x) < 360.0 else 360.0)
traindf["sog"] = traindf["sog"].apply(lambda x: float(x) if float(x) <= 102.2 else 102.3)
traindf["rot"] = traindf["rot"].apply(lambda x: float(x))
traindf["heading"] = traindf["heading"].apply(lambda x: float(x))
traindf["etaRaw"] = traindf.apply(parseEta, axis=1)
traindf["latitude"] = traindf["latitude"].apply(lambda x: float(x))
traindf["longitude"] = traindf["longitude"].apply(lambda x: float(x))

ports.csv

In [12]:
portsdf = pd.read_csv("ports.csv", sep="|")

portsdf["latitude"] = portsdf["latitude"].apply(lambda x: float(x))
portsdf["longitude"] = portsdf["longitude"].apply(lambda x: float(x))

schedules_to_may_2024.csv

In [14]:
schedulesdf = pd.read_csv("~/Ais_Data/schedules_to_may_2024.csv", sep="|", nrows=500) #nrows to speed up action

schedulesdf["arrivalDate"] = schedulesdf["arrivalDate"].apply(lambda x: datetime.fromisoformat(x) if pd.notna(x) else x)
schedulesdf["sailingDate"] = schedulesdf["sailingDate"].apply(lambda x: datetime.fromisoformat(x))
schedulesdf["portLatitude"] = schedulesdf["portLatitude"].apply(lambda x: float(x))
schedulesdf["portLongitude"] = schedulesdf["portLongitude"].apply(lambda x: float(x))

vessels.csv

In [16]:
vehiclesdf = pd.read_csv("vessels.csv", sep="|")

vehiclesdf["CEU"] = vehiclesdf["CEU"].apply(lambda x: int(x) if pd.notna(x) else 0)
vehiclesdf["DWT"] = vehiclesdf["DWT"].apply(lambda x: int(x) if pd.notna(x) else 0)
vehiclesdf["GT"] = vehiclesdf["GT"].apply(lambda x: int(x) if pd.notna(x) else 0)
vehiclesdf["NT"] = vehiclesdf["NT"].apply(lambda x: int(x) if pd.notna(x) else 0)
vehiclesdf["vesselType"] = vehiclesdf["vesselType"].apply(lambda x: int(x) if pd.notna(x) else 0)
vehiclesdf["breadth"] = vehiclesdf["breadth"].apply(lambda x: float(x) if pd.notna(x) else 0)
vehiclesdf["depth"] = vehiclesdf["depth"].apply(lambda x: float(x) if pd.notna(x) else 0)
vehiclesdf["draft"] = vehiclesdf["draft"].apply(lambda x: float(x) if pd.notna(x) else 0)
vehiclesdf["enginePower"] = vehiclesdf["enginePower"].apply(lambda x: float(x) if pd.notna(x) else 0)
vehiclesdf["freshWater"] = vehiclesdf["freshWater"].apply(lambda x: float(x) if pd.notna(x) else 0)
vehiclesdf["fuel"] = vehiclesdf["fuel"].apply(lambda x: float(x) if pd.notna(x) else 0)
vehiclesdf["length"] = vehiclesdf["length"].apply(lambda x: float(x) if pd.notna(x) else 0)
vehiclesdf["maxHeight"] = vehiclesdf["maxHeight"].apply(lambda x: float(x) if pd.notna(x) else 0)
vehiclesdf["maxSpeed"] = vehiclesdf["maxSpeed"].apply(lambda x: float(x) if pd.notna(x) else 0)
vehiclesdf["maxWidth"] = vehiclesdf["maxWidth"].apply(lambda x: float(x) if pd.notna(x) else 0)
vehiclesdf["rampCapacity"] = vehiclesdf["rampCapacity"].apply(lambda x: float(x) if pd.notna(x) else 0)
vehiclesdf["yearBuilt"] = vehiclesdf["yearBuilt"].apply(lambda x: datetime.strptime(str(x), "%Y"))

Function for calculating time difference

In [17]:
def timeDiff(t1: datetime, t2: datetime, i="s"):
    diff = t1 - t2 if t1 >= t2  else t2 - t1
    if i == "s":
        return diff.total_seconds()
    elif i == "m":
        return diff.total_seconds() // 60
    elif i == "h":
        return diff.total_seconds() // 3600
    elif i == "d":
        return diff.days
    else:
        return 0

Adding WORKING_SIZE previous nodes to the ais_train dataframe

In [19]:
def prev_nodes(traindf: pd.DataFrame, vesselId: str, WINDOW_SIZE=1):
    vessel_data = traindf[traindf['vesselId'] == vesselId]
    for i in range(1, len(vessel_data)): #Adding WINDOW_SIZE previous nodes with time diff
        nowTime = vessel_data["time"].iloc[i]
        ind = vessel_data.iloc[i].name

        for j in range(1, WINDOW_SIZE+1):
            if i-j >= 0:
                diff = timeDiff(vessel_data["time"].iloc[i-j], nowTime)
                latitude, longitude = vessel_data["latitude"].iloc[i-j], vessel_data["longitude"].iloc[i-j]
                    
                traindf.loc[ind, "latitude_" + str(j)] = latitude
                traindf.loc[ind, "longitude_" + str(j)] = longitude
                traindf.loc[ind, "Time_" + str(j)] = diff

Checking if a boat is on a schedule

In [20]:
def onSchedule(schedulesdf: pd.DataFrame, vesselId: str, time: datetime):
    vessel_schedules = schedulesdf[schedulesdf['vesselId'] == vesselId]
    fitting_schedule = (vessel_schedules["sailingDate"] >= time) & (vessel_schedules["arrivalDate"] <= time)
    if fitting_schedule.any():
        return vessel_schedules.loc[fitting_schedule.idxmax()]
    else:
        return None

Creating a submission

In [22]:
def submission(sampledf: pd.DataFrame, testdf: pd.DataFrame, deliver=False, name="SampleTest") -> pd.DataFrame:
    sample = sampledf.copy()

    for i in range(len(testdf)):
        vessel_data = traindf[traindf['vesselId'] == testdf.loc[i, "vesselId"]]
        row = vessel_data.sort_values(by="time").iloc[-1]
        sample.loc[i,"longitude_predicted"], sample.loc[i, "latitude_predicted"] = row["longitude"], row["latitude"]
    
    if deliver:
        sample.to_csv(name + ".csv", index=False)
        
    return sample

sample = submission(sampledf, testdf) 
#sample = submission(sampledf, testdf, True, "Test1")